In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sqlalchemy import create_engine

In [3]:
df = pd.read_csv('final_internal_temp.csv')

In [4]:
# We select Temperature and Weather as input value
X = df[['Temperature', 'Weather']]
y = df['Internal Temperature']

In [5]:
# Change categorical variables to binary vectors
X_encoded = pd.get_dummies(X, columns=['Weather'])

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [7]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [8]:
# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [9]:
# Evaluate the model
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

In [10]:
print("Training RMSE:", train_rmse)
print("Testing RMSE:", test_rmse)

Training RMSE: 0.7006963973232718
Testing RMSE: 0.7207520317679105


In [11]:
api_key = "141804de4ea61ac2a29b76576308ed37"
import requests
def get_weather(api_key, city):
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={api_key}"
    response = requests.get(base_url)
    data = response.json()
    if data["cod"] != "404":
        main = data["main"]
        wind = data["wind"]
        weather_desc = data["weather"][0]["description"]

        # collect the necessary data
        temperature = main["temp"]
        feels_like = main["feels_like"]
        humidity = main["humidity"]
        wind_speed = wind["speed"]
        weather = weather_desc.capitalize()

        return temperature, feels_like, humidity, wind_speed, weather
    else:
        return None

# Use your own API Key and preferred city
city = "trento"

weather_data = get_weather(api_key, city)

if weather_data is not None:
    print(f"Temperature: {weather_data[0]}°C")
    print(f"Feels Like: {weather_data[1]}°C")
    print(f"Humidity: {weather_data[2]}%")
    print(f"Wind Speed: {weather_data[3]} m/s")
    print(f"Weather: {weather_data[4]}")
else:
    print("City not found.")

Temperature: 24.6°C
Feels Like: 24.6°C
Humidity: 57%
Wind Speed: 6.17 m/s
Weather: Few clouds


In [12]:
temp = weather_data[0]
weather = weather_data[4]

# Create an initial dataframe
new_data = pd.DataFrame({'Temperature': [temp]})

# Generate a one-hot encoded dataframe for the 'weather' variable
weather_encoded = pd.get_dummies(pd.Series(weather))

# Make sure that it has all the weather types, even if they're not present in this specific instance
for column in X_train.columns:
    if column not in weather_encoded.columns and 'Weather' in column:
        weather_encoded[column] = 0

# Concatenate the temperature dataframe and the one-hot encoded weather dataframe
new_data = pd.concat([new_data, weather_encoded], axis=1)

# Ensure the new_data has the same column order as your training set
new_data = new_data.reindex(columns = X_encoded.columns, fill_value=0)

# Now we can predict using the new data
predictions = model.predict(new_data)

print("Predicted Internal Temperature:", predictions)


Predicted Internal Temperature: [19.67442923]


In [13]:
...
print("Predicted Internal Temperature:", predictions)

import nest_asyncio
from flask import Flask, render_template, jsonify, request
import json

# apply the patch, to enable nested event loops
nest_asyncio.apply()

app = Flask(__name__, template_folder='.')

@app.route("/")
def index():
    return render_template('thermo.html')

# Insert the new "/suggested_temperature" route code here
@app.route("/suggested_temperature", methods=['GET'])
def suggested_temperature():
    weather_data = get_weather(api_key, city)
    if weather_data is None:
        return jsonify({'error': 'City not found.'}), 404

    temp = weather_data[0]
    weather = weather_data[4]

    # Create an initial dataframe
    new_data = pd.DataFrame({'Temperature': [temp]})

    # Generate a one-hot encoded dataframe for the 'weather' variable
    weather_encoded = pd.get_dummies(pd.Series(weather))

    # Make sure that it has all the weather types, even if they're not present in this specific instance
    for column in X_train.columns:
        if column not in weather_encoded.columns and 'Weather' in column:
            weather_encoded[column] = 0

    # Concatenate the temperature dataframe and the one-hot encoded weather dataframe
    new_data = pd.concat([new_data, weather_encoded], axis=1)

    # Ensure the new_data has the same column order as your training set
    new_data = new_data.reindex(columns = X_encoded.columns, fill_value=0)

    # Now we can predict using the new data
    predicted_internal_temp = model.predict(new_data)[0]
    predicted_internal_temp = round(predicted_internal_temp, 2)
    return jsonify({'suggested_temp': predicted_internal_temp})

import threading
# threading.Thread(target=app.run, kwargs={'port':5001}).start()
threading.Thread(target=app.run, kwargs={'port':5001}).start()

Predicted Internal Temperature: [19.67442923]
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2023 18:25:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 18:25:31] "GET /suggested_temperature HTTP/1.1" 200 -


In [13]:
@app.route("/suggested_temperature", methods=['GET'])
def suggested_temperature():
    weather_data = get_weather(api_key, city)
    if weather_data is None:
        return jsonify({'error': 'City not found.'}), 404

    temp = weather_data[0]
    weather = weather_data[4]

    # Create an initial dataframe
    new_data = pd.DataFrame({'Temperature': [temp]})

    # Generate a one-hot encoded dataframe for the 'weather' variable
    weather_encoded = pd.get_dummies(pd.Series(weather))

    # Make sure that it has all the weather types, even if they're not present in this specific instance
    for column in X_train.columns:
        if column not in weather_encoded.columns and 'Weather' in column:
            weather_encoded[column] = 0

    # Concatenate the temperature dataframe and the one-hot encoded weather dataframe
    new_data = pd.concat([new_data, weather_encoded], axis=1)

    # Ensure the new_data has the same column order as your training set
    new_data = new_data.reindex(columns = X_encoded.columns, fill_value=0)

    # Now we can predict using the new data
    predicted_internal_temp = model.predict(new_data)[0]

    return jsonify({'suggested_temp': predicted_internal_temp})


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)


AssertionError: View function mapping is overwriting an existing endpoint function: suggested_temperature

In [ ]:
# Load data from MySQL
# engine = create_engine('your_mysql_connection_string')
# df = pd.read_sql_table('first', engine)
df = pd.read_csv('final_internal_temp.csv')

# Convert categorical variables into dummy/indicator variables
encoder = OneHotEncoder(sparse=False)
weather_encoded = pd.DataFrame(encoder.fit_transform(df[['Weather']]))
weather_encoded.columns = encoder.get_feature_names(['Weather'])

# Drop column 'Weather' as it is now encoded
df = df.drop('Weather', axis = 1)
print(df.head())
# df1 = pd.read_csv('final_internal_temp.csv')

# Join the encoded df
df = df.join(weather_encoded)

WEATHER_COLUMNS = list(weather_encoded.columns)

# Split into prediction target (y) and features (X)
...


# Join the encoded df
# df = df.join(weather_encoded)

# Split into prediction target (y) and features (X)
y = df['Internal Temperature']
features = ['Temperature', 'Feels Like', 'Humidity', 'Wind Speed'] + list(weather_encoded.columns)
X = df[features]

# Split the data into training and validation datasets
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Define the model
model = GradientBoostingRegressor(random_state=0)

# Fit the model
model.fit(train_X, train_y)

# Get predictions
predictions = model.predict(val_X)

# Validate the model using MAE
mae = mean_absolute_error(predictions, val_y)
print(f'Mean Absolute Error: {mae}')


In [ ]:
# def predict_internal_temperature(temp, weather):
#     # weather should be one of the unique weather categories in your data
#     weather_data = [0]*len(WEATHER_COLUMNS)
#     try:
#         weather_data[WEATHER_COLUMNS.index('Weather_' + weather)] = 1
#     except ValueError:
#         print(f"The weather type '{weather}' was not in the original data, thus can't be used for prediction.")
#         return

#     # prepare the feature array
#     features = [temp] + weather_data

#     # make the prediction
#     prediction = model.predict([features])

#     return prediction[0]


In [ ]:
api_key = "141804de4ea61ac2a29b76576308ed37"
import requests
def get_weather(api_key, city):
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={api_key}"
    response = requests.get(base_url)
    data = response.json()
    if data["cod"] != "404":
        main = data["main"]
        wind = data["wind"]
        weather_desc = data["weather"][0]["description"]

        # collect the necessary data
        temperature = main["temp"]
        feels_like = main["feels_like"]
        humidity = main["humidity"]
        wind_speed = wind["speed"]
        weather = weather_desc.capitalize()

        return temperature, feels_like, humidity, wind_speed, weather
    else:
        return None

# Use your own API Key and preferred city
city = "trento"

weather_data = get_weather(api_key, city)

if weather_data is not None:
    print(f"Temperature: {weather_data[0]}°C")
    print(f"Feels Like: {weather_data[1]}°C")
    print(f"Humidity: {weather_data[2]}%")
    print(f"Wind Speed: {weather_data[3]} m/s")
    print(f"Weather: {weather_data[4]}")
else:
    print("City not found.")

In [ ]:
temp = weather_data[0]
feels_like = weather_data[1]
humidity = weather_data[2]
wind_speed = weather_data[3]
weather = "Snowy"  # can be 'Cloudy', 'Rainy', 'Snowy', 'Sunny'

# predicted_internal_temp = predict_internal_temperature(temp, feels_like, humidity, wind_speed, weather)


new_data = pd.DataFrame({'Temperature':temp , 'Weather_Snowy': [0], 'Weather_Rainy': [0], 'Weather_Cloudy': [1], 'Weather_Sunny': [0]})

new_data = new_data[X_encoded.columns]

predictions = model.predict(new_data)

print("Predicted Internal Temperature:", predictions)

In [ ]:
# # let's say it's a sunny day with 25 degrees outside, feels like 27 degrees, 45% humidity, and wind speed 5 m/s
# temp = weather_data[0]
# feels_like = weather_data[1]
# humidity = weather_data[2]
# wind_speed = weather_data[3]
# weather = "Snowy"  # can be 'Cloudy', 'Rainy', 'Snowy', 'Sunny'

# predicted_internal_temp = predict_internal_temperature(temp,weather)

# print(f"The predicted internal temperature is: {predicted_internal_temp}")


In [ ]:
# you need to install the library first
# you can do it by running !pip install nest_asyncio in a Jupyter cell

import nest_asyncio
from flask import Flask, render_template, jsonify, request
import json

# apply the patch, to enable nested event loops
nest_asyncio.apply()

app = Flask(__name__, template_folder='.')

@app.route("/")
def index():
    return render_template('thermo.html')

@app.route("/suggested_temperature", methods=['GET'])
def suggested_temperature():
    temperature = weather_data[0]
    feels_like = weather_data[1]
    humidity = weather_data[2]
    wind_speed = weather_data[3]
    weather = weather_data[4]  # You were referring to weather_data[3] but weather_data is not defined in this context
    return jsonify({'suggested_temp': predictions})  # predicted_internal_temp is not defined in this context

# This will run the app without blocking the notebook
import threading
# threading.Thread(target=app.run, kwargs={'port':5001}).start()
threading.Thread(target=app.run, kwargs={'port':5001}).start()


In [ ]:
from flask import Flask, render_template, jsonify, request
import json

app = Flask(__name__, template_folder='.')

@app.route("/")
def index():
    return render_template('thermo.html')

@app.route("/suggested_temperature", methods=['GET'])
def suggested_temperature():
    temperature = 35
    feels_like = 34
    humidity = 30
    wind_speed = 3.2
    weather = "Snowy"
    suggested_temp = 18
    return jsonify({'suggested_temp': suggested_temp})

if __name__ == "__main__":
    app.run(port=5001, debug=True)